Let's calculate binary and multi class log loss / cross-entropy for a sample dataset; note that this is not a neat implementation of either - I'm just working some things out to understand it better :) 

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics

%matplotlib inline

Step 1: Whip up a sample dataset!

In [11]:
#these will be out predictions
y_pred = pd.DataFrame(np.random.uniform(0,1,10), columns = ["y_pred"])

In [12]:
#these will be the true values of the thing we're predicting
y = pd.DataFrame(np.random.randint(0,2,y_pred.size), columns = ["y"])

In [13]:
df = y_pred.join(y)
df

,y_pred,y
0,0.189906,0
1,0.210304,1
2,0.964080,1
3,0.440199,1
4,0.847321,0
5,0.097608,1
6,0.595879,0
7,0.831450,1
8,0.262923,1
9,0.350548,1


Calculate binary log loss for each row

In [14]:
df['logloss'] = - (df.y * np.log(df.y_pred) + (1 - df.y) * (np.log(1 - df.y_pred)))

In [15]:
df

,y_pred,y,logloss
0,0.189906,0,0.210605
1,0.210304,1,1.559199
2,0.964080,1,0.036581
3,0.440199,1,0.820528
4,0.847321,0,1.879419
5,0.097608,1,2.326798
6,0.595879,0,0.906041
7,0.831450,1,0.184584
8,0.262923,1,1.335893
9,0.350548,1,1.048257


In [16]:
avg_log_loss = df.logloss.mean()
avg_log_loss

1.0307904100792016

Let's compare it with a log loss calculated with sklearn

In [17]:
sklearn.metrics.log_loss(y,y_pred)

1.0307904100792014

######  Multi-class cross-entropy

Let's make an example of a Multi class log loss. In this case, let's prepare a dataset with 4 columns: one for the value of a 3-class thing we want to predict (which takes values from 1 to 3), and 3 variables we'll pretend are our predicted probabilities for each of these 3 clases.

In [18]:
df = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y1"])
df["prob_y2"] = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y2"])
df["prob_y3"] = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y3"])
df["y"] = pd.DataFrame(np.random.randint(1,4,10), columns = ["y"])

In [19]:
df

,prob_y1,prob_y2,prob_y3,y
0,0.837022,0.923960,0.729480,2
1,0.059296,0.023347,0.275162,3
2,0.758702,0.281903,0.710503,2
3,0.697489,0.371361,0.533869,3
4,0.707981,0.866259,0.995853,3
5,0.095159,0.935333,0.128930,2
6,0.080733,0.717363,0.164783,3
7,0.059530,0.516702,0.081829,3
8,0.673730,0.984894,0.138374,2
9,0.517270,0.138448,0.657063,2


Let's normalize the probabilities, so they add up to 1 for each row

In [20]:
sum_of_probs = df[["prob_y1","prob_y2","prob_y3"]].sum(axis=1) 

df["prob_y1"] /= sum_of_probs
df["prob_y2"] /= sum_of_probs
df["prob_y3"] /= sum_of_probs
df

,prob_y1,prob_y2,prob_y3,y
0,0.336091,0.370999,0.292910,2
1,0.165723,0.065251,0.769026,3
2,0.433270,0.160985,0.405745,2
3,0.435191,0.231707,0.333102,3
4,0.275469,0.337054,0.387477,3
5,0.082074,0.806724,0.111202,2
6,0.083845,0.745018,0.171136,3
7,0.090463,0.785188,0.124349,3
8,0.374920,0.548077,0.077003,2
9,0.394026,0.105462,0.500512,2


Let's add separate 'y' columns for each label to make it apparent how multi-class log loss relates to binary log loss

In [21]:
df["y1"] = df["y"] == 1
df["y2"] = df["y"] == 2
df["y3"] = df["y"] == 3
df

,prob_y1,prob_y2,prob_y3,y,y1,y2,y3
0,0.336091,0.370999,0.292910,2,False,True,False
1,0.165723,0.065251,0.769026,3,False,False,True
2,0.433270,0.160985,0.405745,2,False,True,False
3,0.435191,0.231707,0.333102,3,False,False,True
4,0.275469,0.337054,0.387477,3,False,False,True
5,0.082074,0.806724,0.111202,2,False,True,False
6,0.083845,0.745018,0.171136,3,False,False,True
7,0.090463,0.785188,0.124349,3,False,False,True
8,0.374920,0.548077,0.077003,2,False,True,False
9,0.394026,0.105462,0.500512,2,False,True,False


now, it's as simple as taking the negative average of the three binary log losses (without the minus sign)

In [22]:
df['logloss_y1'] = - (df.y1 * np.log(df.prob_y1))
df['logloss_y2'] = - (df.y2 * np.log(df.prob_y2))
df['logloss_y3'] = - (df.y3 * np.log(df.prob_y3))
df['logloss'] = df[["logloss_y1", "logloss_y2", "logloss_y3"]].sum(axis=1)
df

,prob_y1,prob_y2,prob_y3,y,y1,y2,y3,logloss_y1,logloss_y2,logloss_y3,logloss
0,0.336091,0.370999,0.292910,2,False,True,False,0.0,0.991555,0.000000,0.991555
1,0.165723,0.065251,0.769026,3,False,False,True,0.0,0.000000,0.262630,0.262630
2,0.433270,0.160985,0.405745,2,False,True,False,0.0,1.826441,0.000000,1.826441
3,0.435191,0.231707,0.333102,3,False,False,True,0.0,0.000000,1.099306,1.099306
4,0.275469,0.337054,0.387477,3,False,False,True,0.0,0.000000,0.948098,0.948098
5,0.082074,0.806724,0.111202,2,False,True,False,0.0,0.214774,0.000000,0.214774
6,0.083845,0.745018,0.171136,3,False,False,True,0.0,0.000000,1.765296,1.765296
7,0.090463,0.785188,0.124349,3,False,False,True,0.0,0.000000,2.084662,2.084662
8,0.374920,0.548077,0.077003,2,False,True,False,0.0,0.601339,0.000000,0.601339
9,0.394026,0.105462,0.500512,2,False,True,False,0.0,2.249407,0.000000,2.249407


Now we need to sum up all individual losses and take an average of them.

In [23]:
_sum_logloss = df.logloss.sum()

In [24]:
avg_logloss = _sum_logloss/df.shape[0]
avg_logloss

1.204350786499746

In [25]:
sklearn.metrics.log_loss(df[["y1", "y2", "y3"]], df[["prob_y1", "prob_y2", "prob_y3"]])

1.2043507864997463

boom